In [62]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import os
import glob

In [63]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [64]:
path = os.getcwd()  
extension = 'csv'
os.chdir(path)
files_csv = glob.glob('*.{}'.format(extension))

In [65]:
def primeiros_tratamentos(df):
    #tirar umas linhas chatas q estao no final do arquivo
    indice_selecoes = df[df['Capital'] == '{ñ class}'].index[0]
    # Apagar as linhas a partir desse índice até o final
    df = df.iloc[:indice_selecoes]
    #tratando top row que esta deslocada
    df.iloc[0, 1:] = df.iloc[0, :-1].values
    #deletando registros que nao tem informações
    df = df[df[' Faixa Remun Média (SM)'] != '{ñ class}']
    #valores NaN em nulo 
    df = df.replace("NaN", np.nan)
    #preenchendo linhas nan para localidade
    df['Capital'] = df['Capital'].ffill()
    #deletando oq vem antes do nome da capital
    df['Capital'] = df['Capital'].str.split(':').str[1]
    #deletar linhas de totais
    df = df[df[' Faixa Remun Média (SM)'] != 'Total']

    #apagar todas as colunas com total no nome
    df = df.drop(df.filter(like='Total').columns, axis=1)

    #corrigindo header
    df.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df.columns]
    return df 


In [66]:
def criando_df_feminino(df, ano):
    #criando df feminino
    feminino_cols = ['Capital', ' Faixa Remun Média (SM)'] + [coluna for coluna in df.columns if 'Feminino' in coluna]
    df_f=df[feminino_cols]
    #renomeando colunas do que vai ser o novo header
    df_f.iloc[0,0]= "capital"
    df_f.iloc[0,1]= "faixa_remuneracao_media_sm"
    #criando novo header
    new_header = df_f.iloc[0]
    df_f = df_f[1:] 
    df_f.columns = new_header 
    #apagando as colunas duplicadas pq misteriosamente elas duplicam muitas vezes 
    df_f = df_f.loc[:, ~df_f.columns.duplicated()]
    #criand coluna para o sexo feminino
    df_f["sexo"] = "F"
    df_f["ano"] = ano
    #apagar todas as colunas com total no nome
    df_f = df_f.drop(df_f.filter(like='Total').columns, axis=1)
    #corrigindo header
    #df_f.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df_f.columns]
    #df_f.columns = [re.sub(r'[^a-zA-Z0-9]+', '_', unidecode(coluna.lower())) for coluna in df_f.columns]
    df_f = df_f.drop(df_f.filter(like='{ñ class}').columns, axis=1)
    return df_f


In [67]:
feminino_cols = ['Capital', ' Faixa Remun Média (SM)'] + [coluna for coluna in df.columns if 'Feminino' in coluna]
df_f=df[feminino_cols]
df_f[df_f["Capital"].isnull()]

,Capital,Faixa Remun Média (SM),Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino,Feminino
0,NaN,NaN,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total,01:10 A 14,02:15 A 17,03:18 A 24,04:25 A 29,05:30 A 39,06:40 A 49,07:50 A 64,08:65 OU MAIS,{ñ class},Total


In [68]:
def criando_df_masculino(df,ano):
    #criando df Masculino
    masculino_cols = ['Capital', ' Faixa Remun Média (SM)'] + [coluna for coluna in df.columns if 'Masculino' in coluna]
    df_m=df[masculino_cols]
    #renomeando colunas do que vai ser o novo header
    df_m.iloc[0,0]= "capital"
    df_m.iloc[0,1]= "faixa_remuneracao_media_sm"
    #criando novo header
    new_header = df_m.iloc[0]
    df_m = df_m[1:] 
    df_m.columns = new_header 
    #apagando as colunas duplicadas pq misteriosamente elas duplicam muitas vezes 
    df_m = df_m.loc[:, ~df_m.columns.duplicated()]
    #criand coluna para o sexo masculino
    df_m["sexo"] = "M"
    df_m["ano"] = ano
    #apagar todas as colunas com total no nome
    df_m = df_m.drop(df_m.filter(like='Total').columns, axis=1)
    #corrigindo header
    #df_m.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df_m.columns]
    #df_m.columns = [re.sub(r'[^a-zA-Z0-9]+', '_', unidecode(coluna.lower())) for coluna in df_m.columns]
    df_m = df_m.drop(df_m.filter(like='{ñ class}').columns, axis=1)
    return df_m 


In [69]:
df_final  = pd.DataFrame(columns=['capital', 'faixa_remuneracao_media_sm', "01:10 A 14","02:15 A 17","03:18 A 24","04:25 A 29","05:30 A 39","06:40 A 49","07:50 A 64","08:65 OU MAIS",'sexo', 'ano'])

In [70]:
for i in files_csv:

    df = pd.read_csv(i,sep=";",  encoding='latin-1',skiprows=2, header=[0])
    ano = re.search(r'\d{4}', i).group()
    df = primeiros_tratamentos(df)
    df_f = criando_df_feminino(df,ano)
    df_m = criando_df_masculino(df,ano)

    df_final = pd.concat([df_final,df_f, df_m])
    
    print(f"File {i}, com shape {df.shape} do ano {ano}, df feminino: {df_f.shape}, df masculino: {df_m.shape} e df final: {df_final.shape}")



#ULTIMOS TRATAMENTOS:
# Remover acentuação
df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].apply(lambda x: unidecode(str(x)))

# Aplicar expressão regular para extrair os valores desejados
df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].apply(lambda x: re.sub(r'^\d+:(.+)', r'\1', str(x)).strip())

#tirar espaco 
df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].str.replace(' ', '_')


# Transformar em minúsculas
df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].str.lower()
df_final.to_csv('sexoXfaixaetariaXcboXsm.csv', index=False)


    

/Users/bolsolui/Library/Python/3.11/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/bolsolui/Library/Python/3.11/lib/python/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


File sexoXfaixaetariaXcboXsmX2012.csv, com shape (321, 22) do ano 2012, df feminino: (320, 12), df masculino: (320, 12) e df final: (640, 12)
File sexoXfaixaetariaXcboXsmX2006.csv, com shape (322, 22) do ano 2006, df feminino: (321, 12), df masculino: (321, 12) e df final: (1282, 12)
File sexoXfaixaetariaXcboXsmX2007.csv, com shape (322, 22) do ano 2007, df feminino: (321, 12), df masculino: (321, 12) e df final: (1924, 12)
File sexoXfaixaetariaXcboXsmX2013.csv, com shape (324, 22) do ano 2013, df feminino: (323, 12), df masculino: (323, 12) e df final: (2570, 12)
File sexoXfaixaetariaXcboXsmX2005.csv, com shape (321, 22) do ano 2005, df feminino: (320, 12), df masculino: (320, 12) e df final: (3210, 12)
File sexoXfaixaetariaXcboXsmX2011.csv, com shape (322, 22) do ano 2011, df feminino: (321, 12), df masculino: (321, 12) e df final: (3852, 12)
File sexoXfaixaetariaXcboXsmX2010.csv, com shape (323, 22) do ano 2010, df feminino: (322, 12), df masculino: (322, 12) e df final: (4496, 12)


In [71]:
#df= pd.read_csv("sexoXfaixaetariaXcboXsmX2002.csv",sep=";",  encoding='latin-1',skiprows=2, header=[0])
#df= primeiros_tratamentos(df)